# Time windows

## Setup

In [21]:
import numpy as np
import tensorflow as tf

## Time Windows

First, we will train a model to forecast the next step given the previous 20 steps, therefore, we need to create a dataset of 20-step windows for training.

In [2]:
dataset = tf.data.Dataset.range(10)
for val in dataset:
    print(val.numpy())

0
1
2
3
4
5
6
7
8
9


In [5]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1)
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 
6 7 8 9 
7 8 9 
8 9 
9 


In [6]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=2)
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=" ")
    print()

0 1 2 3 4 
2 3 4 5 6 
4 5 6 7 8 
6 7 8 9 
8 9 


In [7]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


In [8]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
    print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


In [9]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
for x, y in dataset:
    print(x.numpy(), y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


In [10]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
for x, y in dataset:
    print(x.numpy(), y.numpy())

[3 4 5 6] [7]
[4 5 6 7] [8]
[0 1 2 3] [4]
[1 2 3 4] [5]
[5 6 7 8] [9]
[2 3 4 5] [6]


In [15]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1)
for x, y in dataset:
    print("x =", x.numpy())
    print("y =", y.numpy())

x = [[5 6 7 8]
 [1 2 3 4]]
y = [[9]
 [5]]
x = [[4 5 6 7]
 [0 1 2 3]]
y = [[8]
 [4]]
x = [[2 3 4 5]
 [3 4 5 6]]
y = [[6]
 [7]]


In [16]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(1).prefetch(1)
for x, y in dataset:
    print("x =", x.numpy())
    print("y =", y.numpy())

x = [[3 4 5 6]]
y = [[7]]
x = [[2 3 4 5]]
y = [[6]]
x = [[4 5 6 7]]
y = [[8]]
x = [[5 6 7 8]]
y = [[9]]
x = [[1 2 3 4]]
y = [[5]]
x = [[0 1 2 3]]
y = [[4]]


In [29]:
def window_dataset(series, window_size, batch_size=32,
                   shuffle_buffer=1000):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [61]:
x = np.linspace(0,1,11)
y = np.sin(x)

In [62]:
len(y)

11

In [63]:
x

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [64]:
window_size = 2
train_set = window_dataset(x, window_size) 


In [65]:
for x, y in train_set:
    print("x =", x.numpy())
    print("y =", y.numpy())

x = [[0.5 0.6]
 [0.1 0.2]
 [0.7 0.8]
 [0.4 0.5]
 [0.  0.1]
 [0.6 0.7]
 [0.8 0.9]
 [0.2 0.3]
 [0.3 0.4]]
y = [0.7 0.3 0.9 0.6 0.2 0.8 1.  0.4 0.5]


In [66]:
list(train_set)

[(<tf.Tensor: shape=(9, 2), dtype=float64, numpy=
  array([[0.1, 0.2],
         [0.4, 0.5],
         [0.5, 0.6],
         [0.3, 0.4],
         [0. , 0.1],
         [0.7, 0.8],
         [0.2, 0.3],
         [0.6, 0.7],
         [0.8, 0.9]])>,
  <tf.Tensor: shape=(9,), dtype=float64, numpy=array([0.3, 0.6, 0.7, 0.5, 0.2, 0.9, 0.4, 0.8, 1. ])>)]